In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv('dataset/흡연자 예측/train.csv')
test = pd.read_csv('dataset/흡연자 예측/test.csv')

In [3]:
train.head(2)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,30.0,180.0,110.0,112.1,1.5,1.5,1.0,1.0,138.0,...,48.0,136.0,16.3,1.0,1.0,27.0,34.0,50.0,1.0,1.0
1,1,35.0,175.0,80.0,87.2,1.0,1.2,1.0,1.0,120.0,...,47.0,127.0,16.0,2.0,1.0,25.0,27.0,28.0,1.0,1.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  int64  
 1   age                  15000 non-null  float64
 2   height(cm)           15000 non-null  float64
 3   weight(kg)           15000 non-null  float64
 4   waist(cm)            15000 non-null  float64
 5   eyesight(left)       15000 non-null  float64
 6   eyesight(right)      15000 non-null  float64
 7   hearing(left)        15000 non-null  float64
 8   hearing(right)       15000 non-null  float64
 9   systolic             15000 non-null  float64
 10  relaxation           15000 non-null  float64
 11  fasting blood sugar  15000 non-null  float64
 12  Cholesterol          15000 non-null  float64
 13  triglyceride         15000 non-null  float64
 14  HDL                  15000 non-null  float64
 15  LDL                  15000 non-null 

In [5]:
test.head(2)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,15000,45.0,160.0,60.0,77.0,1.0,1.0,1.0,1.0,110.0,...,39.0,65.0,83.0,12.7,1.0,0.8,21.0,17.0,18.0,0.0
1,15001,40.0,165.0,75.0,88.0,1.2,1.0,1.0,1.0,120.0,...,202.0,41.0,112.0,14.0,1.0,1.1,20.0,24.0,27.0,1.0


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   height(cm)           10000 non-null  float64
 3   weight(kg)           10000 non-null  float64
 4   waist(cm)            10000 non-null  float64
 5   eyesight(left)       10000 non-null  float64
 6   eyesight(right)      10000 non-null  float64
 7   hearing(left)        10000 non-null  float64
 8   hearing(right)       10000 non-null  float64
 9   systolic             10000 non-null  float64
 10  relaxation           10000 non-null  float64
 11  fasting blood sugar  10000 non-null  float64
 12  Cholesterol          10000 non-null  float64
 13  triglyceride         10000 non-null  float64
 14  HDL                  10000 non-null  float64
 15  LDL                  10000 non-null  

In [7]:
# 결측치 파악

In [8]:
train.isnull().sum().sum()

0

In [9]:
test.isnull().sum().sum()

0

In [10]:
# 데이터 분리

X = train.drop(['id', 'smoking'], axis=1)
y = train['smoking']
test_id = test.pop('id')

In [11]:
# 스케일링

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
test = scaler.transform(test)

In [12]:
# 학습 데이터, 검증 데이터 분리

random_state = 42
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [13]:
# 학습 데이터 모델 피팅

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
# 검증 데이터 추론

preds_proba_val = model.predict_proba(X_val)
preds_proba_val = preds_proba_val[:, 1]

In [15]:
# 검증 데이터 평가
score_val = roc_auc_score(y_val, preds_proba_val)
score_val

0.8762096473391007

In [16]:
# 테스트 데이터 추론

preds_proba_test = model.predict_proba(test)
preds_proba_test = preds_proba_test[:, 1]

In [17]:
# # 제출

# submission = pd.DataFrame({'id': test_id, 'smoking': preds_proba_test})
# submission.to_csv('submission.csv', index=False)